In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import os
from datetime import date
import random 

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import sklearn
import xgboost as xgb
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn import metrics  
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib


import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('expand_frame_repr', True)

In [4]:
g_ignore_features=['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index']

In [5]:
def load_kaggle_csv(filepath, num_rows, root="D:/wangh/Kaggle/HomeCredit/Data"):
    csv_path = os.path.join(root, filepath)
    
    if not os.path.isfile(csv_path) :
            csv_path = os.path.join("E:/SparkExerciseData/Kaggle", filepath)
        
    return pd.read_csv(csv_path, nrows = num_rows)

In [6]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [7]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [8]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = True):
    # Read data and merge
    df = load_kaggle_csv('application_train.csv', num_rows)
    test_df = load_kaggle_csv('application_test.csv', num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    
    # Categorical features: Binary features and One-Hot encoding
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    #AMT_INCOME_TOTAL 117000000 -> nan
    df['AMT_INCOME_TOTAL'].replace(117000000, np.nan, inplace= True)
    
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    del test_df
    gc.collect()
    return df

In [9]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = load_kaggle_csv('bureau.csv', num_rows)
    bb = load_kaggle_csv('bureau_balance.csv', num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(columns= 'SK_ID_BUREAU', inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],
        'AMT_ANNUITY': ['max', 'mean'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACT_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLS_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [10]:
# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = load_kaggle_csv('previous_application.csv', num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APR_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REF_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [11]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = load_kaggle_csv('POS_CASH_balance.csv', num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

In [12]:
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = load_kaggle_csv('installments_payments.csv', num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INS_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

In [13]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = load_kaggle_csv('credit_card_balance.csv', num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(columns = ['SK_ID_PREV'], inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

In [14]:
def print_AUC(alg, input_train, input_Y, input_test, input_test_Y) :
    predictions_train = alg.predict(input_train)
    predictions_prob_train = alg.predict_proba(input_train)[:,1]
    print("Train Accuracy Score:", metrics.accuracy_score(input_Y, predictions_train))
    print("Train AUC Score:", metrics.roc_auc_score(input_Y, predictions_prob_train))

    predictions = alg.predict(input_test)
    predictions_prob = alg.predict_proba(input_test)[:,1]

    print("Accuracy Score:", metrics.accuracy_score(input_test_Y, predictions))
    print("AUC Score:", metrics.roc_auc_score(input_test_Y, predictions_prob))

In [15]:
def cv_alg(alg, cv_df, num_folds=5, stratified = True):
    print("Starting CV {}. CV shape: {}".format(alg.__class__.__name__,cv_df.shape))
    
    global g_ignore_features
    
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
        
    feats = [f for f in cv_df.columns if f not in g_ignore_features]
    
    cv_df_feats = cv_df[feats]
    cv_df_TARGET = cv_df['TARGET']
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(cv_df_feats, cv_df_TARGET)):
        train_x, train_y = cv_df_feats.iloc[train_idx], cv_df_TARGET.iloc[train_idx]
        valid_x, valid_y = cv_df_feats.iloc[valid_idx], cv_df_TARGET.iloc[valid_idx]   
        alg.fit(train_x, train_y)
        print_AUC(alg, train_x, train_y, valid_x,  valid_y)


In [16]:
from sklearn.model_selection import StratifiedShuffleSplit

def fit_alg(alg, fit_train_df, fit_val_df=None):
    print("Starting fit {}. fit shape: {}".format(alg.__class__.__name__,fit_train_df.shape))

    global g_ignore_features
    
    feats = [f for f in fit_train_df.columns if f not in g_ignore_features]
    
    fit_X_train = fit_train_df[feats]
    fit_y_train = fit_train_df['TARGET']
    
    if fit_val_df is None :
        alg.fit(fit_X_train, fit_y_train)
    else :
        fit_X_test = fit_val_df[feats]
        fit_y_test = fit_val_df['TARGET']          
        alg.fit(fit_X_train, fit_y_train)

        print_AUC(alg, fit_X_train, fit_y_train, fit_X_test,  fit_y_test)

In [17]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, clf, num_folds, stratified = False):
    global g_ignore_features
    
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in g_ignore_features]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]


        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 100)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

In [18]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')

# Data preparation

In [19]:
debug = False

In [20]:
#def prepare_data(debug = False):
num_rows = 10000 if debug else None
df = application_train_test(num_rows)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')

with timer("Process previous_applications"):
    prev = previous_applications(num_rows)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')

with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')

with timer("Process installments payments"):
    ins = installments_payments(num_rows)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')

with timer("Process credit card balance"):
    cc = credit_card_balance(num_rows)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')
    
del bureau
del prev
del pos
del ins  
del cc
gc.collect()
#with timer("Run LightGBM with kfold"):
    #feat_importance = kfold_lightgbm(df, num_folds= 5, stratified = False)



Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 122)
Process bureau and bureau_balance - done in 27s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 31s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 16s
Installments payments df shape: (339587, 26)
Process installments payments - done in 35s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 23s


21

In [21]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [22]:
train_df['TARGET'].value_counts()/train_df.shape[0]

0.0    0.919271
1.0    0.080729
Name: TARGET, dtype: float64

In [23]:
import math
ignore_features_median =  ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']
relevant_features_median = [col for col in df.columns if col not in ignore_features_median]
df_imp = df.replace([np.inf, -np.inf], np.nan)

for col in relevant_features_median :
    col_median = df_imp[col].median()
    if math.isnan(col_median) : 
        print("is nan:", col)
        col_median=0.0
    df_imp[col].fillna(col_median, inplace=True)
train_df_imp = df_imp[df_imp['TARGET'].notnull()]
test_df_imp = df_imp[df_imp['TARGET'].isnull()]

In [24]:
df_imp.columns[np.isfinite(df_imp).all()==False]

Index(['TARGET'], dtype='object')

In [25]:
split = StratifiedShuffleSplit(n_splits=1, test_size= 0.3, random_state=1001)
for train_index, val_index in split.split(train_df, train_df['TARGET']):
    X_train, X_val = train_df.loc[train_index], train_df.loc[val_index]
    X_train_imp, X_val_imp = train_df_imp.loc[train_index], train_df_imp.loc[val_index]

In [26]:
train_df['TARGET'].head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: TARGET, dtype: float64

In [27]:
X_train['TARGET'].head()

219233    0.0
41378     0.0
128296    0.0
284923    0.0
271614    0.0
Name: TARGET, dtype: float64

In [28]:
df.shape, X_train.shape, X_val.shape, X_train_imp.shape, X_val_imp.shape

((356255, 816), (215257, 816), (92254, 816), (215257, 816), (92254, 816))

In [29]:
X_train_imp.columns[np.isfinite(X_train_imp).all()==False]

Index([], dtype='object')

In [30]:
train_df[relevant_features_median].iloc[5687:5689, 654:657].head()

,INS_PAYMENT_PERC_MAX,INS_PAYMENT_PERC_MEAN,INS_PAYMENT_PERC_SUM
5687,inf,inf,inf
5688,1.000000,0.857143,12.000000


In [31]:
train_df_imp[relevant_features_median].iloc[5687:5689, 654:657].head()

,INS_PAYMENT_PERC_MAX,INS_PAYMENT_PERC_MEAN,INS_PAYMENT_PERC_SUM
5687,1.0,1.000000,24.617193
5688,1.0,0.857143,12.000000


In [32]:
train_df_imp.head()

,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,HOUR_APPR_PROCESS_START,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_nan,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_nan,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_nan,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_nan,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_nan,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_nan,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_nan,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OC

In [33]:
gc.collect()

0

# Final Models

In [32]:
log_clf = LogisticRegression(penalty='l1', C=1, random_state=1001)

with timer("fit LogisticRegression"):
    fit_alg(log_clf, X_train_imp, X_val_imp)

Starting fit LogisticRegression. fit shape: (215257, 816)
Train Accuracy Score: 0.9199422086157477
Train AUC Score: 0.7806786098495464
Accuracy Score: 0.9190062219524356
AUC Score: 0.7713584983739853
fit LogisticRegression - done in 21832s


In [33]:
rnd_clf = RandomForestClassifier(n_estimators=5000, 
                        max_leaf_nodes=30, 
                        max_features=0.1,
                        max_depth=5,
                        bootstrap=True, n_jobs=-1, random_state=27)
with timer("fit RandomForestClassifier"):
    fit_alg(rnd_clf, X_train_imp, X_val_imp)

Starting fit RandomForestClassifier. fit shape: (215257, 816)
Train Accuracy Score: 0.9192732408237595
Train AUC Score: 0.7531070345256892
Accuracy Score: 0.9192663732737876
AUC Score: 0.7417840647256758
fit RandomForestClassifier - done in 2589s


In [105]:
 lgb_clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=30, max_depth=8,
    learning_rate=0.01, n_estimators=3000,  #10000,  
    objective='binary', min_split_gain=0.02, min_child_weight=40, 
    min_child_samples=500, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.5, reg_lambda=0.8, random_state=27, n_jobs=-1, silent=False
)

with timer("fit lgb.LGBMClassifier"):
    fit_alg(lgb_clf, X_train, X_val)

Starting fit LGBMClassifier. fit shape: (215257, 816)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy Score: 0.925967564353308
Train AUC Score: 0.8816385539163776


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy Score: 0.9199601101307261
AUC Score: 0.7850767103046055
fit lgb.LGBMClassifier - done in 775s


 lgb_clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=30, max_depth=8,
    learning_rate=0.01, n_estimators=3000,  #10000,  
    objective='binary', min_split_gain=0.02, min_child_weight=40, 
    min_child_samples=500, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.5, reg_lambda=0.8, random_state=27, n_jobs=-1, silent=False
)

    
Starting fit LGBMClassifier. fit shape: (215257, 816)

Train Accuracy Score: 0.925967564353308
Train AUC Score: 0.8816385539163776

Accuracy Score: 0.9199601101307261
AUC Score: 0.7850767103046055
fit lgb.LGBMClassifier - done in 775s


#run result
lgb_clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=30, max_depth=8,
    learning_rate=0.01, n_estimators=5000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=27, n_jobs=-1, silent=False
)

with timer("fit lgb.LGBMClassifier"):
    fit_alg(lgb_clf, X_train, X_val)

#result output
Starting fit LGBMClassifier. fit shape: (215257, 816)


Train Accuracy Score: 0.9309197842578871
Train AUC Score: 0.9281868536765366

Accuracy Score: 0.9196891191709845
AUC Score: 0.7841070665789232
fit lgb.LGBMClassifier - done in 1342s



        # LightGBM parameters found by Bayesian optimization
       # clf = LGBMClassifier(
       #     nthread=4,
       #     n_estimators=10000,
       #     learning_rate=0.02,
       #     num_leaves=34,
       #     colsample_bytree=0.9497036,
       #     subsample=0.8715623,
       #     max_depth=8,
       #     reg_alpha=0.041545473,
       #     reg_lambda=0.0735294,
       #     min_split_gain=0.0222415,
       #     min_child_weight=39.3259775,
       #    silent=-1,
       #    verbose=-1, )

In [35]:
xgb_clf = XGBClassifier(
        learning_rate =0.05,
        n_estimators=2000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        objective= 'binary:logistic',
        nthread=8,
        silent=False,
        scale_pos_weight=1,
        seed=27)

with timer("fit XGBClassifier"):
    fit_alg(xgb_clf, X_train, X_val)
    

Starting fit XGBClassifier. fit shape: (215257, 816)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy Score: 0.9385803945980851
Train AUC Score: 0.9531765308983519


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy Score: 0.9192663732737876
AUC Score: 0.7819690354187543
fit XGBClassifier - done in 5406s


#run result
xgb_clf = XGBClassifier(
        learning_rate =0.05,
        n_estimators=2000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        objective= 'binary:logistic',
        nthread=8,
        silent=False,
        scale_pos_weight=1,
        seed=27)

with timer("fit XGBClassifier"):
    fit_alg(xgb_clf, X_train, X_val)
    
#result output
Starting fit XGBClassifier. fit shape: (215257, 816)

Train Accuracy Score: 0.9385803945980851
Train AUC Score: 0.9531765308983519

Accuracy Score: 0.9192663732737876
AUC Score: 0.7819690354187543
fit XGBClassifier - done in 5406s

    

In [36]:
gc.collect()

102

In [36]:
feats = [f for f in test_df.columns if f not in g_ignore_features]
X_val_imp_feats = X_val_imp[feats]
X_val_feats = X_val[feats]


seclvl_train_all = np.concatenate(( np.array([log_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([rnd_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([xgb_clf.predict_proba(X_val_feats)[:,1]]).T,
                                np.array([lgb_clf.predict_proba(X_val_feats)[:,1]]).T
                            ), axis=1)

level2_df = pd.concat([X_val['TARGET'].reset_index(),pd.DataFrame(seclvl_train_all)], axis=1)

In [37]:
gbm_level2 = xgb.XGBClassifier(
 learning_rate = 0.01,
 n_estimators= 1000,
 max_depth=2,
 min_child_weight=80,
 num_leaves=2,   
 gamma=0.1,  
 subsample=0.6, 
 colsample_bytree=1,
 reg_alpha = 0.2,
 objective= 'binary:logistic',
 random_state=1001
 nthread= -1,
 silent=False,
 scale_pos_weight=1)

cv_alg(gbm_level2, level2_df)
fit_alg(gbm_level2, level2_df)

Starting CV XGBClassifier. CV shape: (92254, 6)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy Score: 0.9200157177312268
Train AUC Score: 0.792282311251377


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy Score: 0.9201170604812486
AUC Score: 0.7812430707724887


...

Accuracy Score: 0.9191327913279133
AUC Score: 0.7820151068332047


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy Score: 0.9199907864072407
Train AUC Score: 0.7929110869394198


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy Score: 0.9201084010840108
AUC Score: 0.7774934944382541
Starting fit XGBClassifier. fit shape: (92254, 6)


In [38]:
log_level2 = LogisticRegression(penalty='l2', C=0.001, random_state=1001)

cv_alg(log_level2, level2_df)
fit_alg(log_level2, level2_df)

Starting CV LogisticRegression. CV shape: (92254, 6)
Train Accuracy Score: 0.9192704804747839
Train AUC Score: 0.7889077753934994
Accuracy Score: 0.9192499458053327
AUC Score: 0.7823180753820819
Train Accuracy Score: 0.919271574326247
Train AUC Score: 0.7845096232252045
Accuracy Score: 0.9192455693458349
AUC Score: 0.7999025795063212
Train Accuracy Score: 0.919271574326247
Train AUC Score: 0.7860802733636647
Accuracy Score: 0.9192455693458349
AUC Score: 0.793542390379192
Train Accuracy Score: 0.9192591187469514
Train AUC Score: 0.7886365388881904
Accuracy Score: 0.9192953929539296
AUC Score: 0.7833467280783091
Train Accuracy Score: 0.9192591187469514
Train AUC Score: 0.7897940122465313
Accuracy Score: 0.9192953929539296
AUC Score: 0.7788169984560241
Starting fit LogisticRegression. fit shape: (92254, 6)


### gbm_level2  Submission 7: Kaggle - Your submission scored 0.785,
gbm_level2 - 
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_state=0,
       reg_alpha=0.2, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.6)

gbm_level2 = xgb.XGBClassifier(
 learning_rate = 0.01,
 n_estimators= 1000,
 max_depth=2,
 min_child_weight=1,
 gamma=0.1,  
 subsample=0.6, 
 colsample_bytree=1,
 reg_alpha = 0.2,
 objective= 'binary:logistic',
 nthread= -1,
 silent=False,
 scale_pos_weight=1)
 
#output result
Starting CV XGBClassifier. CV shape: (92254, 6)

Train Accuracy Score: 0.9204086610118967
Train AUC Score: 0.7935299766248891


Accuracy Score: 0.9204422284847171
AUC Score: 0.7813680639471254

Train Accuracy Score: 0.9205587848732436
Train AUC Score: 0.7890711977974596

Accuracy Score: 0.9198959405994255
AUC Score: 0.7989607227635132

Train Accuracy Score: 0.920463937780307
Train AUC Score: 0.7903746991957435

Accuracy Score: 0.9201669286217549
AUC Score: 0.7938135810182776


Train Accuracy Score: 0.920600509457482
Train AUC Score: 0.7930456208663245

Accuracy Score: 0.919620596205962
AUC Score: 0.7825277196384118

Train Accuracy Score: 0.9203159720340361
Train AUC Score: 0.7939689853297902

Accuracy Score: 0.9202168021680217
AUC Score: 0.7783441600647542



In [ ]:
level2_df.head()

In [ ]:
for clf in [lgb_clf, xgb_clf, log_clf, rnd_clf]: 
    joblib.dump(clf, '{}_finalmodel_lvl1.pkl'.format(clf.__class__.__name__))
    
for clf in [gbm_level2, log_level2]: 
    joblib.dump(clf, '{}_finalmodel_lvl2.pkl'.format(clf.__class__.__name__))

# Generate the final result

In [42]:
submission_path= "D:/wangh/Kaggle/HomeCredit/result_submission8_{}{}.csv"

feats = [f for f in test_df.columns if f not in g_ignore_features]

for clf, test_data_df in [
        (lgb_clf, test_df), 
        (xgb_clf, test_df),
        (log_clf, test_df_imp), 
        (rnd_clf, test_df_imp),
    ]: 
    result_df=pd.DataFrame()
    result_df['SK_ID_CURR'] = test_df['SK_ID_CURR']
    result_df['TARGET'] = clf.predict_proba(test_data_df[feats])[:, 1]

    submission_filepath = submission_path.format(clf.__class__.__name__,'_1stlvl')
    result_df.to_csv(submission_filepath, index=False, float_format='%.6f')
    
    del result_df
    gc.collect()


In [44]:
seclvl_test=test_df[feats]

feats = [f for f in test_df.columns if f not in g_ignore_features]
test_df_imp_feats = test_df_imp[feats]
test_df_feats = test_df[feats]


seclvl_train_all = np.concatenate(( np.array([log_clf.predict_proba(test_df_imp_feats)[:,1]]).T,
                                np.array([rnd_clf.predict_proba(test_df_imp_feats)[:,1]]).T,
                                np.array([xgb_clf.predict_proba(test_df_feats)[:,1]]).T,
                                np.array([lgb_clf.predict_proba(test_df_feats)[:,1]]).T
                            ), axis=1)

In [45]:
result_df=pd.DataFrame()
result_df['SK_ID_CURR'] = test_df['SK_ID_CURR']

for clf_level2 in [gbm_level2, log_level2]: 
    result_df['TARGET'] = clf_level2.predict_proba(pd.DataFrame(seclvl_train_all))[:, 1]
    submission_filePath = submission_path.format(clf_level2.__class__.__name__, '_2ndlvl')
    result_df.to_csv(submission_filePath, index=False, float_format='%.6f')

In [46]:
del test_df_imp_feats
del test_df_feats
del result_df
gc.collect()

55

# Tune Models

In [47]:
def grid_search(alg, grid_params, grid_train_df, grid_target_df):
    grid_search_result = GridSearchCV(estimator = alg, param_grid = grid_params, cv=5,
                           scoring='roc_auc', return_train_score=True, verbose=10,n_jobs=8,iid=False)

    grid_search_result.fit(grid_train_df, grid_target_df)

    return grid_search_result

In [52]:
feats = [f for f in train_df.columns if f not in g_ignore_features]

df_feats_imp = train_df_imp[feats]
df_TARGET_imp = train_df_imp['TARGET']

In [ ]:
rnd_clf = RandomForestClassifier(random_state=27)

rnd_param_grid = {
     'n_estimators': [1000,10000, 3000], 'max_features': [0.1, 0.3, 0.5], 
     'min_samples_leaf':range(40, 300, 100), 
    #'max_depth': range(2, 12, 2),
    #'oob_score':[True, False]
}

rnd_grid_search = grid_search(RandomForestClassifier(n_estimators=5000, 
                        max_leaf_nodes=30, 
                        max_features=0.1,
                        max_depth=8,
                        oob_score=True,
                        bootstrap=True, n_jobs=-1, random_state=27), rnd_param_grid, 
                              df_feats_imp, df_TARGET_imp)
rnd_grid_search.grid_scores_, rnd_grid_search.best_params_, rnd_grid_search.best_score_


Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [48]:
feats = [f for f in train_df.columns if f not in g_ignore_features]

df_feats = train_df[feats]
df_TARGET = train_df['TARGET']

In [36]:
lgb_train = lgb.Dataset(data=df_feats, label=df_TARGET)
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 10 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1}
lgb_result=lgb.cv(params, lgb_train, stratified=True, metrics={'auc'}, num_boost_round=5000,
                      nfold=5, early_stopping_rounds=150, verbose_eval=20)
lgb_result

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	cv_agg's auc: 0.738788 + 0.0032822
[40]	cv_agg's auc: 0.739973 + 0.00340554
[60]	cv_agg's auc: 0.738983 + 0.00344697
[80]	cv_agg's auc: 0.741239 + 0.00333113
[100]	cv_agg's auc: 0.742206 + 0.00319856
[120]	cv_agg's auc: 0.744196 + 0.00326907
[140]	cv_agg's auc: 0.745702 + 0.00348815
[160]	cv_agg's auc: 0.747244 + 0.00351115
[180]	cv_agg's auc: 0.748612 + 0.00343666
[200]	cv_agg's auc: 0.74952 + 0.00351955
[220]	cv_agg's auc: 0.750603 + 0.00346048
[240]	cv_agg's auc: 0.751364 + 0.00342972
[260]	cv_agg's auc: 0.753008 + 0.00346648
[280]	cv_agg's auc: 0.754261 + 0.00346019
[300]	cv_agg's auc: 0.755862 + 0.00339991
[320]	cv_agg's auc: 0.757563 + 0.00336221
[340]	cv_agg's auc: 0.759196 + 0.00324852
[360]	cv_agg's auc: 0.760702 + 0.00314682
[380]	cv_agg's auc: 0.762442 + 0.00311959
[400]	cv_agg's auc: 0.764085 + 0.00314281
[420]	cv_agg's auc: 0.765534 + 0.00310879
[440]	cv_agg's auc: 0.766975 + 0.00309674
[460]	cv_agg's auc: 0.768285 + 0.00304864
[480]	cv_agg's auc: 0.769466 + 0.0030253

{'auc-mean': [0.7098111705909742,
  0.7157302683080881,
  0.7170503297075597,
  0.7246847297570228,
  0.7262433121331824,
  0.726747290818077,
  0.7326943372085626,
  0.7322340334081654,
  0.7328961480092154,
  0.7326242919376217,
  0.7351581047600073,
  0.7372818092061357,
  0.7375229462292385,
  0.739094462061417,
  0.738927433454214,
  0.739648604178577,
  0.7393220475369393,
  0.7393918591306334,
  0.7391273715632914,
  0.7387877031887892,
  0.7385077401543307,
  0.7382617872000904,
  0.7379318503844499,
  0.7383259331546659,
  0.7381204838321392,
  0.7388020826220811,
  0.7384739098381267,
  0.7381965659460974,
  0.738011384570836,
  0.7382617508855476,
  0.7385780212007548,
  0.7394006846309263,
  0.7399008548743767,
  0.739699977876266,
  0.7395265378213799,
  0.7393058793372187,
  0.7395473807655911,
  0.7397265665705879,
  0.7401500689118371,
  0.7399725764694212,
  0.7398086883620619,
  0.7396657630344213,
  0.7398136923824858,
  0.7396666315524925,
  0.7395593718274093,
  0.

In [37]:
len(lgb_result['auc-mean'])

2849

In [39]:
#grid search
lgb_param_test1 = {
    'max_depth':[8], #range(2,10,2),
    'num_leaves':[30], #range(30,60,10),
    'min_child_samples':range(200,1000,300),
    'min_child_weight':range(20,100,20)
    
}
lgb_gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=len(lgb_result['auc-mean']),  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=1001, n_jobs=-1, silent=False
), param_grid = lgb_param_test1, scoring='roc_auc',n_jobs=4,verbose=10,iid=False, cv=5)


lgb_gsearch1.fit(df_feats, df_TARGET, eval_metric='auc', verbose = 50)
lgb_gsearch1.grid_scores_, lgb_gsearch1.best_params_, lgb_gsearch1.best_score_ 

SyntaxError: invalid syntax (<ipython-input-39-e989b700b78a>, line 6)

In [ ]:
def modelfit(alg, train, train_target, useTrainCV=True, cv_folds=5, early_stopping_rounds=150, val_percent=0.2):
    
    split = StratifiedShuffleSplit(n_splits=1, test_size= val_percent, random_state=1001)
    for train_index, test_index in split.split(train, train_target):
        dtrain, dtest = train.loc[train_index], train.loc[test_index]
        dtrain_target, dtest_target = train_target.loc[train_index], train_target.loc[test_index]
            
            
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=dtrain_target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics={'auc'}, early_stopping_rounds=early_stopping_rounds)
        print("n_estimators use : %d" % cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, dtrain_target,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
        
    #Print model report:
    print("\ndtrain Model Report")
    print("dtrain Accuracy : %.4g" % metrics.accuracy_score(dtrain_target, dtrain_predictions))
    print("dtrain AUC Score: %f" % metrics.roc_auc_score(dtrain_target, dtrain_predprob)) 
    
    #     Predict on validation data:
    dtest_predictions = alg.predict(dtest)
    dtest_predprob = alg.predict_proba(dtest)[:,1]
        
    #Print model report:
    print("\ndtest Model Report")
    print("dtest Accuracy : %.4g" % metrics.accuracy_score(dtest_target, dtest_predictions))
    print("dtest AUC Score: %f" % metrics.roc_auc_score(dtest_target, dtest_predprob))

In [ ]:
xgb1 = xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        silent=False,    
        seed=1001)
modelfit(xgb1, df_feats, df_TARGET)

In [ ]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, 
                                                  n_estimators=xgb1.get_params()['n_estimators'], 
                                        max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, 
                                                         random_state=1001, silent=False,  seed=1001, verbose=10), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,verbose=10,iid=False, cv=5)
gsearch1.fit(df_feats, df_TARGET)

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=xgb1.get_params()['n_estimators'],
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,
                                                  random_state=27, silent=False, seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False,verbose=10, cv=5)
gsearch3.fit(df_feats, df_TARGET)

In [ ]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=1000, 
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=gsearch3.best_params_['gamma'], 
                                        subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=2, verbose=10,silent=False,
                                                  scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, verbose=10,cv=5)
gsearch4.fit(df_feats, df_TARGET)

In [ ]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=232,
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=gsearch3.best_params_['gamma'],  
                                                  subsample=gsearch4.best_params_["subsample"], 
                                        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
                                                  silent=False,
                                        objective= 'binary:logistic', nthread=2, verbose=10,
                                                  scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(df_feats, df_TARGET)

In [ ]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
xgb3 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth==gsearch1.best_params_['max_depth'],
        min_child_weight=gsearch1.best_params_['min_child_weight'],
        gamma=gsearch3.best_params_['gamma'], 
        subsample=gsearch4.best_params_["subsample"],
        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
        reg_alpha=gsearch6.best_params_["reg_alpha"],
        objective= 'binary:logistic',
        nthread=8,
        scale_pos_weight=1,
        silent=False,
        seed=27)
modelfit(xgb3, df_feats, df_TARGET)

In [ ]:
xgb3

In [ ]:
xgb4 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=gsearch1.best_params_['max_depth'],
        min_child_weight=gsearch1.best_params_['min_child_weight'],
        gamma=gsearch3.best_params_['gamma'],
        subsample=gsearch4.best_params_["subsample"],
        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
        reg_alpha=gsearch6.best_params_["reg_alpha"],
        objective= 'binary:logistic',
        nthread=8,
        silent=False,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb4, df_feats, df_TARGET)

In [ ]:
xgb4

In [97]:
feats = [f for f in test_df.columns if f not in g_ignore_features]
X_val_imp_feats = X_val_imp[feats]
X_val_feats = X_val[feats]

grid_seclvl_train_all = np.concatenate(( np.array([log_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([rnd_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([xgb_clf.predict_proba(X_val_feats)[:,1]]).T,
                                np.array([lgb_clf.predict_proba(X_val_feats)[:,1]]).T
                            ), axis=1)

grid_level2_df = pd.concat([X_val['TARGET'].reset_index(),pd.DataFrame(grid_seclvl_train_all)], axis=1)

feats_level2 = [f for f in grid_level2_df.columns if f not in g_ignore_features]
level2_df_feats = grid_level2_df[feats_level2]
level2_df_TARGET = grid_level2_df['TARGET']

In [98]:
level2_df_feats.head()

,0,1,2,3
0,0.097511,0.075185,0.119020,0.115630
1,0.044863,0.077860,0.037171,0.050541
2,0.032726,0.059147,0.073451,0.044999
3,0.022680,0.054122,0.019203,0.010699
4,0.362191,0.182466,0.503716,0.562541


In [84]:
log_param_grid_level2 = {'penalty':['l1','l2'],'C': [0.001, 0.01, 0.1, 1, 2, 10] }
log_grid_search_level2 = grid_search(LogisticRegression(), log_param_grid_level2,
                                     level2_df_feats, level2_df_TARGET)

log_grid_search_level2.grid_scores_, log_grid_search_level2.best_params_, log_grid_search_level2.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   16.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   16.9s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   17.8s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   18.6s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   19.8s
[Parallel(n_jobs=8)]: Done  52 out of  60 | elapsed:   20.5s remaining:    3.1s
[Parallel(n_jobs=8)]: Done  60 out of  60 | elapsed:   20.9s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.50000, std: 0.00000, params: {'C': 0.001, 'penalty': 'l1'},
  mean: 0.78756, std: 0.00168, params: {'C': 0.001, 'penalty': 'l2'},
  mean: 0.78754, std: 0.00159, params: {'C': 0.01, 'penalty': 'l1'},
  mean: 0.78755, std: 0.00178, params: {'C': 0.01, 'penalty': 'l2'},
  mean: 0.78660, std: 0.00214, params: {'C': 0.1, 'penalty': 'l1'},
  mean: 0.78719, std: 0.00194, params: {'C': 0.1, 'penalty': 'l2'},
  mean: 0.78462, std: 0.00259, params: {'C': 1, 'penalty': 'l1'},
  mean: 0.78553, std: 0.00237, params: {'C': 1, 'penalty': 'l2'},
  mean: 0.78448, std: 0.00262, params: {'C': 2, 'penalty': 'l1'},
  mean: 0.78504, std: 0.00248, params: {'C': 2, 'penalty': 'l2'},
  mean: 0.78438, std: 0.00263, params: {'C': 10, 'penalty': 'l1'},
  mean: 0.78450, std: 0.00261, params: {'C': 10, 'penalty': 'l2'}],
 {'C': 0.001, 'penalty': 'l2'},
 0.7875643917280424)

log_param_grid_level2 = {'penalty':['l1','l2'],'C': [0.001, 0.01, 0.1, 1, 2, 10] }
log_grid_search_level2 = grid_search(LogisticRegression(), log_param_grid_level2,
                                     level2_df_feats, level2_df_TARGET)

log_grid_search_level2.grid_scores_, log_grid_search_level2.best_params_, log_grid_search_level2.best_score_
([mean: 0.50000, std: 0.00000, params: {'C': 0.001, 'penalty': 'l1'},
  mean: 0.78756, std: 0.00168, params: {'C': 0.001, 'penalty': 'l2'},
  mean: 0.78754, std: 0.00159, params: {'C': 0.01, 'penalty': 'l1'},
  mean: 0.78755, std: 0.00178, params: {'C': 0.01, 'penalty': 'l2'},
  mean: 0.78660, std: 0.00214, params: {'C': 0.1, 'penalty': 'l1'},
  mean: 0.78719, std: 0.00194, params: {'C': 0.1, 'penalty': 'l2'},
  mean: 0.78462, std: 0.00259, params: {'C': 1, 'penalty': 'l1'},
  mean: 0.78553, std: 0.00237, params: {'C': 1, 'penalty': 'l2'},
  mean: 0.78448, std: 0.00262, params: {'C': 2, 'penalty': 'l1'},
  mean: 0.78504, std: 0.00248, params: {'C': 2, 'penalty': 'l2'},
  mean: 0.78438, std: 0.00263, params: {'C': 10, 'penalty': 'l1'},
  mean: 0.78450, std: 0.00261, params: {'C': 10, 'penalty': 'l2'}],
 {'C': 0.001, 'penalty': 'l2'},
 0.7875643917280424)

In [88]:
#grid search
lgb_param_level2 = {
    'n_estimators': range (1000,3000, 1000),
    'max_depth':range(2,10,2),
    'num_leaves':range(30,60,10),
    'min_child_samples':range(200,1000,300),
    'min_child_weigh':range(20,100,20)
}

lgb_gsearch1_level2 = grid_search(lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=2000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=27, n_jobs=-1, silent=False), 
    lgb_param_level2, level2_df_feats, level2_df_TARGET)
lgb_gsearch1_level2.grid_scores_, lgb_gsearch1_level2.best_params_, lgb_gsearch1_level2.best_score_

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    7.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   27.7s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   44.0s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  2.1min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  3.6min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  4.1min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  4.8min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  5.6min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  6.2min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  7.1min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  8.0min
[Parallel(n_jobs=8)]: Do

([mean: 0.78689, std: 0.00126, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 1000, 'num_leaves': 30},
  mean: 0.78689, std: 0.00126, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 1000, 'num_leaves': 40},
  mean: 0.78689, std: 0.00126, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 1000, 'num_leaves': 50},
  mean: 0.78645, std: 0.00140, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 2000, 'num_leaves': 30},
  mean: 0.78645, std: 0.00140, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 2000, 'num_leaves': 40},
  mean: 0.78645, std: 0.00140, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 20, 'n_estimators': 2000, 'num_leaves': 50},
  mean: 0.78689, std: 0.00126, params: {'max_depth': 2, 'min_child_samples': 200, 'min_child_weigh': 40, 'n_estimators': 100

In [92]:
#grid search
lgb_param_level2 = {
    'n_estimators': [1000], #range (1000,3000, 1000),
    'max_depth': [2], #range(2,10,2),
    'num_leaves': [10, 20, 30], #range(30,60,10),
    'min_child_samples': 500, # range(200,1000,300),
    'min_child_weight':range(20,100,20),
    'reg_alpha': [i/10.0 for i in range(0,5),
    'reg_lambda': [i/10.0 for i in range(0,5)]
}

lgb_gsearch1_level2 = grid_search(lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=2000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=27, n_jobs=-1, silent=False), 
    lgb_param_level2, level2_df_feats, level2_df_TARGET)
lgb_gsearch1_level2.grid_scores_, lgb_gsearch1_level2.best_params_, lgb_gsearch1_level2.best_score_

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    7.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   27.5s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   37.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   49.4s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  3.4min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  3.8min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  4.8min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  5.3min
[Parallel(n_jobs=8)]: Do

([mean: 0.78670, std: 0.00152, params: {'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 20, 'n_estimators': 1000, 'num_leaves': 10, 'reg_alpha': 0.0, 'reg_lambda': 0.0},
  mean: 0.78671, std: 0.00153, params: {'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 20, 'n_estimators': 1000, 'num_leaves': 10, 'reg_alpha': 0.0, 'reg_lambda': 0.1},
  mean: 0.78671, std: 0.00152, params: {'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 20, 'n_estimators': 1000, 'num_leaves': 10, 'reg_alpha': 0.0, 'reg_lambda': 0.2},
  mean: 0.78671, std: 0.00151, params: {'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 20, 'n_estimators': 1000, 'num_leaves': 10, 'reg_alpha': 0.0, 'reg_lambda': 0.3},
  mean: 0.78671, std: 0.00150, params: {'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 20, 'n_estimators': 1000, 'num_leaves': 10, 'reg_alpha': 0.0, 'reg_lambda': 0.4},
  mean: 0.78671, std: 0.00151, params: {'max_depth': 2, 'min_child_samples': 500

In [102]:
#grid search
lgb_param_level2 = {
    'n_estimators': [1000], #range (1000,3000, 1000),
    'max_depth': [2], #range(2,10,2),
    'num_leaves': [2,5,8,10], #[10, 20, 30], #range(30,60,10),
    'min_child_samples': [500], # range(200,1000,300),
    'min_child_weight': [80], #range(20,100,20),
    'reg_alpha':[0.1], #[i/10.0 for i in range(0,5)],
    'reg_lambda': [0.2], #[i/10.0 for i in range(0,5)],
    'subsample': [0.6, 0.8, 0.85, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

lgb_gsearch1_level2 = grid_search(lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=2000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.2, random_state=27, n_jobs=-1, silent=False), 
    lgb_param_level2, level2_df_feats, level2_df_TARGET)
lgb_gsearch1_level2.grid_scores_, lgb_gsearch1_level2.best_params_, lgb_gsearch1_level2.best_score_

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    6.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   22.5s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   29.8s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   41.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   52.5s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  3.0min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  3.4min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  3.8min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  4.2min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  4.8min
[Parallel(n_jobs=8)]: Do

([mean: 0.78680, std: 0.00156, params: {'colsample_bytree': 0.8, 'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 80, 'n_estimators': 1000, 'num_leaves': 2, 'reg_alpha': 0.1, 'reg_lambda': 0.2, 'subsample': 0.6},
  mean: 0.78680, std: 0.00156, params: {'colsample_bytree': 0.8, 'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 80, 'n_estimators': 1000, 'num_leaves': 2, 'reg_alpha': 0.1, 'reg_lambda': 0.2, 'subsample': 0.8},
  mean: 0.78680, std: 0.00156, params: {'colsample_bytree': 0.8, 'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 80, 'n_estimators': 1000, 'num_leaves': 2, 'reg_alpha': 0.1, 'reg_lambda': 0.2, 'subsample': 0.85},
  mean: 0.78680, std: 0.00156, params: {'colsample_bytree': 0.8, 'max_depth': 2, 'min_child_samples': 500, 'min_child_weight': 80, 'n_estimators': 1000, 'num_leaves': 2, 'reg_alpha': 0.1, 'reg_lambda': 0.2, 'subsample': 1.0},
  mean: 0.78675, std: 0.00152, params: {'colsample_bytree': 0.8, 'max_depth': 2, 'min_child_sam

#selected parameter
#CV - 0.7868105299054646

lgb_gsearch1_level2 = grid_search(lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=2000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.2, random_state=27, n_jobs=-1, silent=False), 
    lgb_param_level2, level2_df_feats, level2_df_TARGET)

gb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=2, max_depth=2,
    learning_rate=0.01, n_estimators=1000,  
    objective='binary', min_split_gain=0.02, min_child_weight=80, 
    min_child_samples=500, subsample=0.6, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.2, random_state=27, n_jobs=-1, silent=False)
    
 {'colsample_bytree': 1.0,
  'max_depth': 2,
  'min_child_samples': 500,
  'min_child_weight': 80,
  'n_estimators': 1000,
  'num_leaves': 2,
  'reg_alpha': 0.1,
  'reg_lambda': 0.2,
  'subsample': 0.6},
 0.7868105299054646)

In [ ]:
xgb_param_level2 = {
    'n_estimators':1000, # range (1000, 10000, 2000),
    'max_depth':[2, 3], #range(3,10,2),
    'min_child_weight':3, #range(1,6,2),
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)]
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]    
}

xgb_gsearch_level2 = grid_search(XGBClassifier(learning_rate =0.1, n_estimators=1000,
                                                  max_depth=6,
                                        min_child_weight=1,
                                                  gamma=0, subsample=0.8, colsample_bytree=1,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,
                                                  random_state=27, silent=False, seed=27), 
    xgb_param_level2, level2_df_feats, level2_df_TARGET)
xgb_gsearch_level2.grid_scores_, xgb_gsearch_level2.best_params_, xgb_gsearch_level2.best_score_

xgb_param_level2 = {
    'n_estimators': range (1000, 10000, 2000),
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2),
    #'gamma':[i/10.0 for i in range(0,5)],
    #'subsample':[i/10.0 for i in range(6,10)],
    #'colsample_bytree':[i/10.0 for i in range(6,10)]
    #'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]    
}

xgb_gsearch_level2 = grid_search(XGBClassifier(learning_rate =0.1, n_estimators=1000,
                                                  max_depth=6,
                                        min_child_weight=1,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,
                                                  random_state=27, silent=False, seed=27), 
    xgb_param_level2, level2_df_feats, level2_df_TARGET)
xgb_gsearch_level2.grid_scores_, xgb_gsearch_level2.best_params_, xgb_gsearch_level2.best_score_

#result output
([mean: 0.77955, std: 0.00186, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 1000},
  mean: 0.76449, std: 0.00424, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 3000},
  mean: 0.75526, std: 0.00490, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 5000},
  mean: 0.74743, std: 0.00631, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 7000},
  mean: 0.74146, std: 0.00688, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 9000},
  mean: 0.77977, std: 0.00152, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 1000},
  mean: 0.76607, std: 0.00297, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 3000},
  mean: 0.75533, std: 0.00463, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 5000},
  mean: 0.74719, std: 0.00569, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 7000},
  mean: 0.74193, std: 0.00617, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 9000},
  mean: 0.77959, std: 0.00148, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 1000},
  mean: 0.76560, std: 0.00330, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 3000},
  mean: 0.75571, std: 0.00444, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 5000},
  mean: 0.74800, std: 0.00550, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 7000},
  mean: 0.74263, std: 0.00587, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 9000},
  mean: 0.76887, std: 0.00314, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 1000},
  mean: 0.74589, std: 0.00566, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 3000},
  mean: 0.73536, std: 0.00731, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 5000},
  mean: 0.72754, std: 0.00788, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 7000},
  mean: 0.72150, std: 0.00810, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 9000},
  mean: 0.76788, std: 0.00376, params: {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 1000},
  mean: 0.74411, std: 0.00583, params: {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 3000},
  mean: 0.73179, std: 0.00684, params: {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 5000},
  mean: 0.72395, std: 0.00695, params: {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 7000},
  mean: 0.71811, std: 0.00713, params: {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 9000},
  mean: 0.76738, std: 0.00285, params: {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 1000},
  mean: 0.74422, std: 0.00521, params: {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 3000},
  mean: 0.73283, std: 0.00634, params: {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 5000},
  mean: 0.72481, std: 0.00689, params: {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 7000},
  mean: 0.71887, std: 0.00680, params: {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 9000},
  mean: 0.75709, std: 0.00508, params: {'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 1000},
  mean: 0.73335, std: 0.00650, params: {'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 3000},
  mean: 0.72100, std: 0.00742, params: {'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 5000},
  mean: 0.71306, std: 0.00798, params: {'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 7000},
  mean: 0.70680, std: 0.00805, params: {'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 9000},
  mean: 0.75488, std: 0.00498, params: {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 1000},
  mean: 0.72896, std: 0.00596, params: {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 3000},
  mean: 0.71684, std: 0.00643, params: {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 5000},
  mean: 0.70897, std: 0.00723, params: {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 7000},
  mean: 0.70363, std: 0.00728, params: {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 9000},
  mean: 0.75537, std: 0.00477, params: {'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 1000},
  mean: 0.72882, std: 0.00649, params: {'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 3000},
  mean: 0.71708, std: 0.00695, params: {'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 5000},
  mean: 0.70945, std: 0.00769, params: {'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 7000},
  mean: 0.70383, std: 0.00813, params: {'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 9000},
  mean: 0.74826, std: 0.00543, params: {'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 1000},
  mean: 0.72195, std: 0.00650, params: {'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 3000},
  mean: 0.70971, std: 0.00715, params: {'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 5000},
  mean: 0.70295, std: 0.00768, params: {'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 7000},
  mean: 0.69831, std: 0.00789, params: {'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 9000},
  mean: 0.74505, std: 0.00501, params: {'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 1000},
  mean: 0.71821, std: 0.00645, params: {'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 3000},
  mean: 0.70638, std: 0.00715, params: {'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 5000},
  mean: 0.70005, std: 0.00767, params: {'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 7000},
  mean: 0.69577, std: 0.00786, params: {'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 9000},
  mean: 0.74644, std: 0.00673, params: {'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 1000},
  mean: 0.71899, std: 0.00710, params: {'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 3000},
  mean: 0.70694, std: 0.00756, params: {'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 5000},
  mean: 0.70030, std: 0.00798, params: {'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 7000},
  mean: 0.69562, std: 0.00852, params: {'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 9000}],
 {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 1000},
 0.7797689162770496)

# Load models

In [34]:
model_pkl_path="C:/Users/wangh/Kaggle/model_pkl"

In [35]:
lgb_clf = joblib.load(os.path.join(model_pkl_path, "LGBMClassifier_finalmodel_lvl1.pkl"))
xgb_clf = joblib.load(os.path.join(model_pkl_path, "XGBClassifier_finalmodel_lvl1.pkl"))
log_clf = joblib.load(os.path.join(model_pkl_path, "LogisticRegression_finalmodel_lvl1.pkl"))
rnd_clf = joblib.load(os.path.join(model_pkl_path, "RandomForestClassifier_finalmodel_lvl1.pkl"))